<a href="https://colab.research.google.com/github/nicolashernandez/MdL-2024-2025/blob/main/model_parameters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercice : Nombre de paramètres d'un modèle

En lisant les papiers sources et éventuellement en jetant un oeil aux implémentations, êtes-vous à même de vérifier que le nombre de paramètres de BERT_BASE [1] est de 110 millions de paramètres et que GPT-2 [2] est bien de 1,5 milliards de paramètres ?

* [1] https://github.com/google-research/bert et Devlin, J., Chang, M. W., Lee, K., & Toutanova, K. (2018). Bert: Pre-training of deep bidirectional transformers for language understanding. arXiv preprint arXiv:1810.04805.
* [2] https://github.com/openai/gpt-2 et https://openai.com/index/gpt-2-1-5b-release/

Le code suivant initie un modèle BERT implémenté en PyTorch et interfacé par la bibliothèque `transformers` de huggingface.

In [ ]:
from transformers import BertTokenizer, BertModel
bert_model = BertModel.from_pretrained("bert-base-uncased")

Un affichage du modèle via l'instruction `print` permet d'expliciter ses composants, leurs combinaisons ainsi que la `shape` (taille) des tenseurs.


In [ ]:
print (model)

## Question 1 : taille du modèle BERT

**TODO** Calculer la taille du modèle en appliquant les principes suivants
* le nombre de paramètre d'une couche linéaire est égal au produit de $features_{in} * features_{out} + param_{bias}$ où $param_{bias}$ correspond à features_{out} si `bias=True`
* sommer le nombre de paramètres des couches adjacentes
* multiplier le nombre de paramètres par le facteur de combinaison quand les couches sont déclarées en liste

Garder la trace du total de nombre de paramètres par composant. Vous pouvez utiliser un [tableur](https://lite.framacalc.org) pour ce faire.


L'instruction `summary` du package [torch-summary](https://pypi.org/project/torch-summary/) résume un modèle PyTorch donné en parammètre. Les informations du résumé inclut:
1. Layer names,
2. input/output shapes,
3. kernel shape,
4. number of parameters,
5. number of operations (Mult-Adds)


In [ ]:
!pip install torch-summary

In [ ]:
from torchsummary import summary

summary(model)

**TODO** Obtenez-vous la même chose ? Sinon dans quelle couche observez-vous des différences ?

# Exercice : Generative sampling techniques


## Question 1 : Temperature sampling

La technique dite de l'échantillonage par température ($t$) permet de lisser (température haute) la distribution des probabilités sur toutes les options ou de concentrer (température basse) la distribution  sur les options les plus probables. Le paramètre $t$ s'applique avant que la distribution soit calculée.

Ci-dessous la fonction softmax qui permet de calculer la distribution donné x.

$\sigma(x_i) = \frac{e^{x_{i}/t}}{\sum_{j=1}^K e^{x_{j}/t}} \ \ \ pour\ i=1,2,\dots,K$ et avec temperature $t=1$, pour le softmax par défaut

**TODO** implémenter la fonction softmax à l'aide des fonction numpy exp et sum.

Soit $x$ un vecteur avec des scores associés à 4 mots désignés par A, B, C et D tel que $x=[1.5, -1.8, 0.9, -3.2]$. Vous devriez retrouvez les résultats suivants :

* t=1, softmax = `[0.62704177 0.02312729 0.34412782 0.00570312]`
* t=0.5, softmax = `[7.67673394e-01 1.04431835e-03 2.31218783e-01 6.35050642e-05]`

Si vous ne trouvez pas, jetez un oeil à la documention de [softmax sous scipy](https://docs.scipy.org/doc/scipy/reference/generated/scipy.special.softmax.html).

In [ ]:
import numpy as np

def softmax(x, t=1):
  """TODO"""
    return None


x = np.array([1.5, -1.8, 0.9, -3.2])
print(softmax(x))
print(softmax(x, t=0.5))

**TODO** Observez la distribution suivant les températures. Quelle valeur de $t$ rend le système quasi-déterministe ?

In [ ]:
import pandas as pd

temperatures =  [5, 2, 1, 0.5, 0.1, 0.01]

distribution = list()
for t in temperatures:
  distribution.append(softmax(x, t=t))

df = pd.DataFrame(distribution, columns=['A', 'B', 'C', 'D'], index=temperatures)
print (df)

ax = df.plot.bar(stacked=True)
ax.set_xlabel("Temperature")
ax.set_ylabel("Probability")

## Question 2 : "Top K Sampling"

Pour cette question et la suivante vous utiliserez la démo suivante https://artefact2.github.io/llm-sampling/index.xhtml à la fois pour
* observer les valeurs de la distribution initiale
* vérifier que vos calculs sont corrects

Cela implique de ne pas jouer tout de suite avec la démo...

Sélectionner le contexte suivant "*You will pay for what you have done," she hissed, her blade flashing in the moonlight. The battle that ensued _____*"




**TODO** Considérer la technique "Top K", calculer "à la main" les nouveaux scores de probabilités si K=3. Vérifier après coup.

## Question 3 : "Top P Sampling"

Conserver le même contexte que précédemment sur l'espace démo.

La technique d'échantillonage avec le Top P implique de garder les tokens les plus probables jusqu'à ce que la somme de leurs probabilités soit plus grande que P.

**TODO** Considérer la technique "Top P", combien de mots seront considérés si P=0.8. Calculer les nouveaux scores de probabilité de ces mots.

# La loi "empirique" de Chinchilla

La loi de Chinchilla observée dabs [Training Compute-Optimal Large Language Models (Hoffmann et al. 2022)](https://arxiv.org/abs/2203.15556) recommande d'utiliser 20 fois plus de tokens d'entraînement que le nombre de paramètres du modèle construit.


## Question : les modèles actuels

Quelle est la tendance des modèles actuels ?

La "Table 3" de la communication [A Comprehensive Overview of Large Language Models (Naveed et al, 2024)](https://arxiv.org/abs/2307.06435) indique le nombre de paramètres (*No. of Params*) et le nombre de tokens d'entraînement utilisés (*Data/Tokens*) pour de nombreux modèles récents.

**TODO** Calculer le nombre de tokens d'entraînement théorique requis selon la loi pour les modèles les plus récents à savoir LLaMA-3.1, Snowflake Arctic, Nemotron-4 340B et DeepSeek-v2. En comparant avec le nombre de tokens réels utilisés, observez-vous une tendance chez les modèles récents ?



| Name             | Nu. of parameters | Chinchilla's pretraining tokens | Real pretraining tokens |
|------------------|-------------------|---------------------------------|-------------------------|
| LLaMA-3.1        |                   |                                 |                         |
| Snowflake Arctic |                   |                                 |                         |
| Nemotron-4 340B  |                   |                                 |                         |
| DeepSeek-v2      |                   |                                 |                         |
